In [ ]:
import os
import numpy as np
from sklearn.cluster import KMeans
from sklearn import metrics
import pandas as pd

# Charger les données pour les vraies catégories
file_path = os.path.join("..", "data", "preprocessed_product_data.csv")
data = pd.read_csv(file_path)

# Charger les embeddings textuels (TF-IDF)
text_embedding_file = os.path.join("..", "data", "tfidf_embeddings.npz")
text_data = np.load(text_embedding_file)
text_embeddings = text_data['embeddings']
product_ids_text = text_data['product_id']

# Charger les embeddings visuels (VGG16 - modèle 2)
visual_embedding_file = os.path.join("..", "models", "best_model2_vgg16_embeddings.npz")  # Chemin vers les embeddings VGG16 modèle 2
visual_data = np.load(visual_embedding_file)
visual_embeddings = visual_data['embeddings']
product_ids_visual = visual_data['product_id']

# Vérifier que les identifiants de produits sont dans le même ordre (ou les réaligner)
assert np.array_equal(product_ids_text, product_ids_visual), "Les identifiants de produits ne correspondent pas entre les deux embeddings !"

# Concaténer les embeddings textuels et visuels
combined_embeddings = np.concatenate([text_embeddings, visual_embeddings], axis=1)

# Appliquer KMeans sur les embeddings combinés
num_clusters = len(np.unique(data['category_num']))  # Nombre de clusters selon les catégories réelles
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
kmeans.fit(combined_embeddings)

# Labels prédits par KMeans
predicted_labels = kmeans.labels_

# Vrais labels (catégories réelles)
true_labels = data['category_num'].values  # Les vraies catégories doivent être dans votre dataframe

# Calculer le score ARI (Adjusted Rand Index)
ari_score = metrics.adjusted_rand_score(true_labels, predicted_labels)
print(f"ARI Score for combined text + image embeddings: {ari_score:.4f}")

# Enregistrer les embeddings concaténés
output_file_combined = os.path.join("..", "data", "combined_text_image_embeddings.npz")
np.savez_compressed(output_file_combined, product_id=product_ids_text, embeddings=combined_embeddings)
print(f"Combined embeddings have been saved to: {output_file_combined}")
